In [1]:
import pandas as pd
import numpy as np
import re


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
CSV_PATH = r"/content/drive/MyDrive/DataSet/tamil.csv"

df = pd.read_csv(CSV_PATH, encoding="utf-8")
print("Loaded shape:", df.shape)
df.head()


Loaded shape: (20000, 7)


,original_title,category,full_news,sub_article,semantic_title,lexical_title,random_title
0,“புதிய மொழி கற்றால் தமிழ் அழியுமா? தமிழ் மொழிய...,Politics,விழுப்புரம் : ”புதிய கல்வி கொள்கை தாய் மொழியை ...,கம்பன் வடமொழியை கற்றுக்கொண்டதால் தான் ராமகாவிய...,“திமுக இல்லையென்றால் தமிழும் தமிழகமும் இருக்கா...,‘பேய்’ பொம்மையை கைது செய்த போலீஸார்: மெக்சிகோவ...,மண்டைக்காடு கோயில் வரும் பெண் பக்தர்கள் சிரமம்...
1,ரெப்போ வட்டி விகிதம் 0.25 சதவீதம் உயர்வு - வீ...,Business,மும்பை: ரிசர்வ் வங்கி கவர்னர் சக்திகாந்த தாஸ் ...,மும்பை: ரிசர்வ் வங்கி கவர்னர் சக்திகாந்த தாஸ் ...,ஏர்டெல் ப்ரீபெய்ட் கட்டணம் 25% வரை உயர்வு: எந...,கரோனா தொற்று; குணமடைவோர் விகிதம் 73 சதவீதமாக உ...,Happy Birthday Hardik Pandya: பவர்ஃபுல் ஹிட்டர...
2,ரிஷி சுனக் மனைவிக்கு ஒரே நாளில் ரூ.500 கோடி இழ...,World,லண்டன்: இன்போசிஸ் நிறுவனத்தின் பங்கு மதிப்பு ந...,அக்‌ஷதா மூர்த்திக்கு இன்போசிஸ் நிறுவனத்தில் 09...,டிஜிட்டல் தனிநபர் தகவல் பாதுகாப்பு விதிமீறினால...,ஒருநாள் கிரிக்கெட்டில் பாகிஸ்தானுக்கு 500-வது ...,சென்னை பெண்ணிடம் அமெரிக்க மருத்துவர் என ஏமாற்ற...
3,மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்கெட் - தெ...,Sports,பெனோனி: மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்க...,பெனோனி: மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்க...,அண்டர்-19 மகளிர் T20 WC | இறுதிப் போட்டிக்கு ம...,சென்னையில் இன்று உலகக் கோப்பை ஸ்குவாஷ் தொடக்கம்,ஆஷஸ் 2-வது டெஸ்ட் | ஸ்மித் நிதான ஆட்டம்: முதல்...
4,IPL 2023 | ஹர்ஷல் படேலின் மன்கட் முயற்சி: அஸ்வ...,Sports,சென்னை: நடப்பு ஐபிஎல் சீசனின் 15-வது லீக் போட்...,ஒரு பந்து தான் எஞ்சி உள்ளது நான்-ஸ்ட்ரைக்கர் எ...,நான் இறுதிப் போட்டியில் விளையாட விரும்பினேன் -...,தும்பிக்கை போல் படிக்கட்டுகள்!\nஉச்சிப்பிள்ளைய...,9 வியாழக்கிழமைகள்; நாலுபேருக்கு உணவுப்பொட்டலம்...


In [5]:
required_cols = ["full_news", "semantic_title"]

missing = [c for c in required_cols if c not in df.columns]
print("Missing columns:", missing)
print("All columns:", list(df.columns))


Missing columns: []
All columns: ['original_title', 'category', 'full_news', 'sub_article', 'semantic_title', 'lexical_title', 'random_title']


In [6]:
df = df[["full_news", "semantic_title"]].copy()
print("Selected columns shape:", df.shape)
df.head()


Selected columns shape: (20000, 2)


,full_news,semantic_title
0,விழுப்புரம் : ”புதிய கல்வி கொள்கை தாய் மொழியை ...,“திமுக இல்லையென்றால் தமிழும் தமிழகமும் இருக்கா...
1,மும்பை: ரிசர்வ் வங்கி கவர்னர் சக்திகாந்த தாஸ் ...,ஏர்டெல் ப்ரீபெய்ட் கட்டணம் 25% வரை உயர்வு: எந...
2,லண்டன்: இன்போசிஸ் நிறுவனத்தின் பங்கு மதிப்பு ந...,டிஜிட்டல் தனிநபர் தகவல் பாதுகாப்பு விதிமீறினால...
3,பெனோனி: மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்க...,அண்டர்-19 மகளிர் T20 WC | இறுதிப் போட்டிக்கு ம...
4,சென்னை: நடப்பு ஐபிஎல் சீசனின் 15-வது லீக் போட்...,நான் இறுதிப் போட்டியில் விளையாட விரும்பினேன் -...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2970 (\N{TAMIL LETTER CA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Matplotlib currently does not support Tamil natively.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3014 (\N{TAMIL VOWEL SIGN E}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2985 (\N{TAMIL LETTER NNNA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3021 (\N{TAMIL SIGN VIRAMA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3016 (\N{TAMIL VOWEL SIGN AI}) missing from font(s) Deja

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('full_news').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['semantic_title'].value_counts()
    for x_label, grp in _df_6.groupby('full_news')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('full_news')
_ = plt.ylabel('semantic_title')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2997 (\N{TAMIL LETTER VA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Matplotlib currently does not support Tamil natively.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3007 (\N{TAMIL VOWEL SIGN I}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2996 (\N{TAMIL LETTER LLLA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3009 (\N{TAMIL VOWEL SIGN U}) miss

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_7['full_news'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_7, x='index', y='full_news', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2980 (\N{TAMIL LETTER TA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Matplotlib currently does not support Tamil natively.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3007 (\N{TAMIL VOWEL SIGN I}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 2990 (\N{TAMIL LETTER MA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 3009 (\N{TAMIL VOWEL SIGN U}) missin

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['semantic_title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='semantic_title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [8]:
print("NaN counts:")
print(df.isna().sum())


NaN counts:
full_news         92
semantic_title    19
dtype: int64


In [9]:
before = len(df)
df = df.dropna(subset=["full_news", "semantic_title"]).reset_index(drop=True)
after = len(df)
print("Removed NaN rows:", before - after)
print("Now shape:", df.shape)


Removed NaN rows: 92
Now shape: (19908, 2)


In [10]:
empty_news = (df["full_news"].astype(str).str.strip() == "").sum()
empty_title = (df["semantic_title"].astype(str).str.strip() == "").sum()

print("Empty full_news rows:", empty_news)
print("Empty semantic_title rows:", empty_title)


Empty full_news rows: 0
Empty semantic_title rows: 0


In [11]:
dup_count = df.duplicated().sum()
print("Exact duplicate rows:", dup_count)


Exact duplicate rows: 1687


In [12]:
before = len(df)
df = df.drop_duplicates().reset_index(drop=True)
after = len(df)

print("Removed duplicates:", before - after)
print("Now shape:", df.shape)


Removed duplicates: 1687
Now shape: (18221, 2)


In [13]:
dup_news = df.duplicated(subset=["full_news"]).sum()
print("Duplicate full_news:", dup_news)


Duplicate full_news: 13


In [14]:
before = len(df)
df = df.drop_duplicates(subset=["full_news"], keep="first").reset_index(drop=True)
after = len(df)

print("Removed duplicate full_news:", before - after)
print("Now shape:", df.shape)


Removed duplicate full_news: 13
Now shape: (18208, 2)


In [15]:
df["news_len"] = df["full_news"].astype(str).str.len()
df["title_len"] = df["semantic_title"].astype(str).str.len()

df[["news_len", "title_len"]].describe()


,news_len,title_len
count,18208.000000,18208.000000
mean,1967.667619,76.034710
std,1284.116734,24.629918
min,30.000000,8.000000
25%,1165.000000,59.000000
50%,1639.000000,77.000000
75%,2366.000000,93.000000
max,20329.000000,182.000000


In [16]:
before = len(df)

df = df[(df["news_len"] >= 50) & (df["title_len"] >= 5)].reset_index(drop=True)

after = len(df)
print("Removed short rows:", before - after)
print("Now shape:", df.shape)


Removed short rows: 3
Now shape: (18205, 4)


In [17]:
junk_rows = df["full_news"].astype(str).str.contains(r"<.*?>|http|www|&nbsp;", regex=True, na=False).sum()
print("Rows containing HTML/URL/junk:", junk_rows)

df[df["full_news"].astype(str).str.contains(r"<.*?>|http|www|&nbsp;", regex=True, na=False)].head(5)


Rows containing HTML/URL/junk: 645


,full_news,semantic_title,news_len,title_len
22,செய்திகளைத் தெரிந்துகொள்ள மேலும் ஒரு புதிய வழி...,தளம் புதிது: உங்களுக்காக ஒரு இணையதளம்,925,37
35,உலகில் இதுவரை எத்தனை போர்கள் நடைபெற்றிருக்கும்...,கணை ஏவு காலம் 17 | பக்கத்து வீட்டுப் புரட்சி @...,1186,90
83,நடப்பு டி20 உலகக் கோப்பை தொடரில் சாம்பியன் பட்...,T20 WC | ஆஸ்திரேலிய அணியில் கேமரூன் கிரீன்,1333,42
95,நொய்டா: இந்திய ஸ்மார்ட்போன் சந்தையில் லாவா பிள...,குறைந்த விலையில் லாவா ஸ்மார்ட்போன் அறிமுகம்,839,43
124,டப்ளின்: இந்தியா - அயர்லாந்து அணிகள் இடையிலான ...,WI vs IND | மழையால் 2வது டெஸ்ட் டிரா: மே.இ தீவ...,1770,92


In [18]:
def clean_text(t):
    t = str(t)
    t = re.sub(r"<.*?>", " ", t)       # remove HTML
    t = re.sub(r"http\S+", " ", t)     # remove URLs
    t = re.sub(r"&nbsp;", " ", t)      # remove &nbsp;
    t = re.sub(r"\s+", " ", t)         # collapse spaces
    return t.strip()

df["full_news"] = df["full_news"].apply(clean_text)
df["semantic_title"] = df["semantic_title"].apply(clean_text)

print("Cleaned text.")
df.head()


Cleaned text.


,full_news,semantic_title,news_len,title_len
0,விழுப்புரம் : ”புதிய கல்வி கொள்கை தாய் மொழியை ...,“திமுக இல்லையென்றால் தமிழும் தமிழகமும் இருக்கா...,1982,79
1,மும்பை: ரிசர்வ் வங்கி கவர்னர் சக்திகாந்த தாஸ் ...,ஏர்டெல் ப்ரீபெய்ட் கட்டணம் 25% வரை உயர்வு: எந்...,2211,83
2,லண்டன்: இன்போசிஸ் நிறுவனத்தின் பங்கு மதிப்பு ந...,டிஜிட்டல் தனிநபர் தகவல் பாதுகாப்பு விதிமீறினால...,589,67
3,பெனோனி: மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்க...,அண்டர்-19 மகளிர் T20 WC | இறுதிப் போட்டிக்கு ம...,1990,67
4,சென்னை: நடப்பு ஐபிஎல் சீசனின் 15-வது லீக் போட்...,நான் இறுதிப் போட்டியில் விளையாட விரும்பினேன் -...,1779,66


In [19]:
print("Final NaN counts:\n", df.isna().sum())
print("Final empty news:", (df["full_news"].str.strip() == "").sum())
print("Final empty title:", (df["semantic_title"].str.strip() == "").sum())
print("Final exact duplicates:", df.duplicated().sum())
print("Final shape:", df.shape)


Final NaN counts:
 full_news         0
semantic_title    0
news_len          0
title_len         0
dtype: int64
Final empty news: 0
Final empty title: 0
Final exact duplicates: 0
Final shape: (18205, 4)


In [20]:
df_final = df[["full_news", "semantic_title"]].copy()
df_final.head()


,full_news,semantic_title
0,விழுப்புரம் : ”புதிய கல்வி கொள்கை தாய் மொழியை ...,“திமுக இல்லையென்றால் தமிழும் தமிழகமும் இருக்கா...
1,மும்பை: ரிசர்வ் வங்கி கவர்னர் சக்திகாந்த தாஸ் ...,ஏர்டெல் ப்ரீபெய்ட் கட்டணம் 25% வரை உயர்வு: எந்...
2,லண்டன்: இன்போசிஸ் நிறுவனத்தின் பங்கு மதிப்பு ந...,டிஜிட்டல் தனிநபர் தகவல் பாதுகாப்பு விதிமீறினால...
3,பெனோனி: மகளிருக்கான யு 19 உலகக் கோப்பை கிரிக்க...,அண்டர்-19 மகளிர் T20 WC | இறுதிப் போட்டிக்கு ம...
4,சென்னை: நடப்பு ஐபிஎல் சீசனின் 15-வது லீக் போட்...,நான் இறுதிப் போட்டியில் விளையாட விரும்பினேன் -...


In [21]:
SAVE_PATH = "/content/drive/MyDrive/DataSet/tamil_clean.csv"
df_final.to_csv(SAVE_PATH, index=False, encoding="utf-8")


In [22]:
import json

JSONL_PATH = "/content/drive/MyDrive/DataSet/tamil_clean.jsonl"

with open(JSONL_PATH, "w", encoding="utf-8") as f:
    for _, row in df_final.iterrows():
        f.write(json.dumps({
            "instruction": "கீழே உள்ள செய்தியை சுருக்கமாக எழுதுங்கள்.",
            "input": row["full_news"],
            "output": row["semantic_title"]
        }, ensure_ascii=False) + "\n")
